# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fbac03d2460>
├── 'a' --> tensor([[-1.9692, -1.4157, -0.1254],
│                   [-1.0452, -0.8637,  1.1003]])
└── 'x' --> <FastTreeValue 0x7fbac03d23a0>
    └── 'c' --> tensor([[-0.3311, -0.6493, -0.9600, -0.2359],
                        [ 2.0764, -0.8973, -2.5271, -0.6419],
                        [ 0.3659, -1.2581,  0.4015,  0.5354]])

In [4]:
t.a

tensor([[-1.9692, -1.4157, -0.1254],
        [-1.0452, -0.8637,  1.1003]])

In [5]:
%timeit t.a

77.2 ns ± 0.135 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fbac03d2460>
├── 'a' --> tensor([[ 0.7377,  0.1620, -1.3687],
│                   [ 0.3111, -0.6096, -0.0665]])
└── 'x' --> <FastTreeValue 0x7fbac03d23a0>
    └── 'c' --> tensor([[-0.3311, -0.6493, -0.9600, -0.2359],
                        [ 2.0764, -0.8973, -2.5271, -0.6419],
                        [ 0.3659, -1.2581,  0.4015,  0.5354]])

In [7]:
%timeit t.a = new_value

81.8 ns ± 0.145 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.9692, -1.4157, -0.1254],
               [-1.0452, -0.8637,  1.1003]]),
    x: Batch(
           c: tensor([[-0.3311, -0.6493, -0.9600, -0.2359],
                      [ 2.0764, -0.8973, -2.5271, -0.6419],
                      [ 0.3659, -1.2581,  0.4015,  0.5354]]),
       ),
)

In [10]:
b.a

tensor([[-1.9692, -1.4157, -0.1254],
        [-1.0452, -0.8637,  1.1003]])

In [11]:
%timeit b.a

70.3 ns ± 0.185 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.8147,  1.2047, -1.0598],
               [-0.5690,  0.3768,  1.0704]]),
    x: Batch(
           c: tensor([[-0.3311, -0.6493, -0.9600, -0.2359],
                      [ 2.0764, -0.8973, -2.5271, -0.6419],
                      [ 0.3659, -1.2581,  0.4015,  0.5354]]),
       ),
)

In [13]:
%timeit b.a = new_value

588 ns ± 1.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

983 ns ± 1.77 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.2 µs ± 59.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

171 µs ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

169 µs ± 808 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fba141b8fd0>
├── 'a' --> tensor([[[-1.9692, -1.4157, -0.1254],
│                    [-1.0452, -0.8637,  1.1003]],
│           
│                   [[-1.9692, -1.4157, -0.1254],
│                    [-1.0452, -0.8637,  1.1003]],
│           
│                   [[-1.9692, -1.4157, -0.1254],
│                    [-1.0452, -0.8637,  1.1003]],
│           
│                   [[-1.9692, -1.4157, -0.1254],
│                    [-1.0452, -0.8637,  1.1003]],
│           
│                   [[-1.9692, -1.4157, -0.1254],
│                    [-1.0452, -0.8637,  1.1003]],
│           
│                   [[-1.9692, -1.4157, -0.1254],
│                    [-1.0452, -0.8637,  1.1003]],
│           
│                   [[-1.9692, -1.4157, -0.1254],
│                    [-1.0452, -0.8637,  1.1003]],
│           
│                   [[-1.9692, -1.4157, -0.1254],
│                    [-1.0452, -0.8637,  1.1003]]])
└── 'x' --> <FastTreeValue 0x7fba141b8c70>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

38.1 µs ± 145 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fba140d40a0>
├── 'a' --> tensor([[-1.9692, -1.4157, -0.1254],
│                   [-1.0452, -0.8637,  1.1003],
│                   [-1.9692, -1.4157, -0.1254],
│                   [-1.0452, -0.8637,  1.1003],
│                   [-1.9692, -1.4157, -0.1254],
│                   [-1.0452, -0.8637,  1.1003],
│                   [-1.9692, -1.4157, -0.1254],
│                   [-1.0452, -0.8637,  1.1003],
│                   [-1.9692, -1.4157, -0.1254],
│                   [-1.0452, -0.8637,  1.1003],
│                   [-1.9692, -1.4157, -0.1254],
│                   [-1.0452, -0.8637,  1.1003],
│                   [-1.9692, -1.4157, -0.1254],
│                   [-1.0452, -0.8637,  1.1003],
│                   [-1.9692, -1.4157, -0.1254],
│                   [-1.0452, -0.8637,  1.1003]])
└── 'x' --> <FastTreeValue 0x7fbac03d2580>
    └── 'c' --> tensor([[-0.3311, -0.6493, -0.9600, -0.2359],
                        [ 2.0764, -0.8973, -2.5271, -0.6419],
                 

In [23]:
%timeit t_cat(trees)

35.4 µs ± 174 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

64.7 µs ± 1.62 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.9692, -1.4157, -0.1254],
                [-1.0452, -0.8637,  1.1003]],
       
               [[-1.9692, -1.4157, -0.1254],
                [-1.0452, -0.8637,  1.1003]],
       
               [[-1.9692, -1.4157, -0.1254],
                [-1.0452, -0.8637,  1.1003]],
       
               [[-1.9692, -1.4157, -0.1254],
                [-1.0452, -0.8637,  1.1003]],
       
               [[-1.9692, -1.4157, -0.1254],
                [-1.0452, -0.8637,  1.1003]],
       
               [[-1.9692, -1.4157, -0.1254],
                [-1.0452, -0.8637,  1.1003]],
       
               [[-1.9692, -1.4157, -0.1254],
                [-1.0452, -0.8637,  1.1003]],
       
               [[-1.9692, -1.4157, -0.1254],
                [-1.0452, -0.8637,  1.1003]]]),
    x: Batch(
           c: tensor([[[-0.3311, -0.6493, -0.9600, -0.2359],
                       [ 2.0764, -0.8973, -2.5271, -0.6419],
                       [ 0.3659, -1.2581,  0.4015,  0.5354]],
         

In [26]:
%timeit Batch.stack(batches)

90.3 µs ± 301 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.9692, -1.4157, -0.1254],
               [-1.0452, -0.8637,  1.1003],
               [-1.9692, -1.4157, -0.1254],
               [-1.0452, -0.8637,  1.1003],
               [-1.9692, -1.4157, -0.1254],
               [-1.0452, -0.8637,  1.1003],
               [-1.9692, -1.4157, -0.1254],
               [-1.0452, -0.8637,  1.1003],
               [-1.9692, -1.4157, -0.1254],
               [-1.0452, -0.8637,  1.1003],
               [-1.9692, -1.4157, -0.1254],
               [-1.0452, -0.8637,  1.1003],
               [-1.9692, -1.4157, -0.1254],
               [-1.0452, -0.8637,  1.1003],
               [-1.9692, -1.4157, -0.1254],
               [-1.0452, -0.8637,  1.1003]]),
    x: Batch(
           c: tensor([[-0.3311, -0.6493, -0.9600, -0.2359],
                      [ 2.0764, -0.8973, -2.5271, -0.6419],
                      [ 0.3659, -1.2581,  0.4015,  0.5354],
                      [-0.3311, -0.6493, -0.9600, -0.2359],
                      [ 2.0764, -

In [28]:
%timeit Batch.cat(batches)

176 µs ± 6.18 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

387 µs ± 2.85 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
